# EDA - Online Retail

Exploracion y limpieza del dataset de transacciones (2010-2011) antes de la fase de mineria de patrones.


## Objetivos
- Cargar el dataset de forma reproducible (local o Kaggle)
- Analizar datos faltantes y calidad (cancelaciones, valores invalidos, duplicados)
- Limpiar y documentar la perdida de filas/productos
- Explorar estacionalidad y productos principales
- Guardar un CSV procesado listo para mineria


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

sns.set_theme(style="whitegrid")
pd.options.display.float_format = "{:,.2f}".format


## Rutas y configuracion


In [ ]:
DATA_DIR = Path("data")
RAW_PATH = DATA_DIR / "raw" / "online_retail.csv"
PROCESSED_PATH = DATA_DIR / "processed" / "cleaned_online_retail.csv"

for p in [RAW_PATH.parent, PROCESSED_PATH.parent]:
    p.mkdir(parents=True, exist_ok=True)

SEED = 42
np.random.seed(SEED)


## Carga de datos
Busca `data/raw/online_retail.csv`. Si no existe y hay credenciales de Kaggle configuradas, lo descarga via `kagglehub`.


In [ ]:
def load_data(raw_path=RAW_PATH):
    if raw_path.exists():
        df = pd.read_csv(raw_path)
        source = raw_path
    else:
        try:
            import kagglehub
            path = Path(kagglehub.dataset_download("ulrikthygepedersen/online-retail-dataset"))
            src = path / "online_retail.csv"
            if not src.exists():
                raise FileNotFoundError(f"No se encontro online_retail.csv en {src}")
            shutil.copy(src, raw_path)
            df = pd.read_csv(raw_path)
            source = raw_path
        except Exception as exc:
            raise RuntimeError("Coloca online_retail.csv en data/raw/ o configura Kaggle") from exc
    return df, source


df_raw, source = load_data()
print(f"Fuente de datos: {source}")
print(f"Filas: {len(df_raw):,} | Columnas: {df_raw.shape[1]}")
df_raw.head()


## Perfil inicial


In [ ]:
display(df_raw.describe(include='all'))
print("
Tipos de dato:")
print(df_raw.dtypes)


## Datos faltantes y calidad


In [ ]:
missing = df_raw.isna().mean().sort_values(ascending=False)
missing = (missing * 100).round(2).to_frame(name="pct_missing")
missing


In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(x=missing.index, y=missing['pct_missing'], palette='mako')
plt.xticks(rotation=45)
plt.ylabel('% faltante')
plt.title('Porcentaje de valores faltantes por columna')
plt.tight_layout()
plt.show()


In [ ]:
quality_checks = {
    "cancelaciones": df_raw['InvoiceNo'].astype(str).str.startswith('C').sum(),
    "cantidades_no_positivas": (df_raw['Quantity'] <= 0).sum(),
    "precios_no_positivos": (df_raw['UnitPrice'] <= 0).sum(),
    "descripciones_vacias": df_raw['Description'].isna().sum() + (df_raw['Description'].astype(str).str.strip() == '').sum(),
    "duplicados_exactos": df_raw.duplicated().sum(),
}
pd.DataFrame.from_dict(quality_checks, orient='index', columns=['conteo'])


## Limpieza con pipeline
Pasos:
1. Eliminar cancelaciones (`InvoiceNo` que empieza con "C").
2. Mantener `Quantity` y `UnitPrice` positivos.
3. Quitar descripciones vacias/NA.
4. Convertir tipos y eliminar duplicados exactos.
5. Filtrar productos poco frecuentes (default: >=50 apariciones) para reducir sparsity.
6. Recortar outliers via IQR (se recorta, no se elimina fila) para estabilizar distribuciones.


In [ ]:
def clip_iqr(series, factor=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    return series.clip(lower, upper)


def clean_retail(df, min_item_freq=50):
    log = []
    out = df.copy()
    out['InvoiceNo'] = out['InvoiceNo'].astype(str).str.strip()
    start_len = len(out)

    def log_step(name, before, after):
        log.append({
            "paso": name,
            "filas_removidas": before - after,
            "pct_inicial": (before - after) / start_len * 100
        })

    before = len(out)
    out = out[~out['InvoiceNo'].str.startswith('C')]
    log_step("Eliminar cancelaciones", before, len(out))

    before = len(out)
    out = out[(out['Quantity'] > 0) & (out['UnitPrice'] > 0)]
    log_step("Cantidad y precio positivos", before, len(out))

    out['Description'] = out['Description'].astype(str).str.strip()
    before = len(out)
    out = out[out['Description'] != ""]
    out = out.dropna(subset=['Description'])
    log_step("Descripciones vacias/NA", before, len(out))

    out['InvoiceDate'] = pd.to_datetime(out['InvoiceDate'])
    out['Country'] = out['Country'].astype(str).str.strip()

    before = len(out)
    out = out.drop_duplicates()
    log_step("Duplicados exactos", before, len(out))

    counts = out['Description'].value_counts()
    keep_items = counts[counts >= min_item_freq].index
    before = len(out)
    out = out[out['Description'].isin(keep_items)]
    log_step(f"Items con freq >= {min_item_freq}", before, len(out))

    out['Quantity'] = clip_iqr(out['Quantity'])
    out['UnitPrice'] = clip_iqr(out['UnitPrice'])

    summary = {
        "filas_iniciales": start_len,
        "filas_finales": len(out),
        "productos": out['Description'].nunique(),
        "facturas": out['InvoiceNo'].nunique(),
        "clientes": out['CustomerID'].nunique(),
        "rango_fechas": (out['InvoiceDate'].min(), out['InvoiceDate'].max())
    }

    return out, pd.DataFrame(log), summary


df, log_df, summary = clean_retail(df_raw, min_item_freq=50)
log_df


In [ ]:
pd.DataFrame([summary])


## Distribuciones y outliers (recortados)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,4))
sns.boxplot(y=df['Quantity'], ax=axes[0])
axes[0].set_title('Quantity (recortado)')
sns.boxplot(y=df['UnitPrice'], ax=axes[1])
axes[1].set_title('UnitPrice (recortado)')
plt.tight_layout()
plt.show()


## Estacionalidad de ventas (mensual)


In [ ]:
monthly_qty = df.set_index('InvoiceDate').resample('M')['Quantity'].sum()
changes = monthly_qty.pct_change()
spikes = changes[changes.abs() > 0.25]

plt.figure(figsize=(10, 4))
ax = sns.lineplot(x=monthly_qty.index, y=monthly_qty.values, marker='o')
plt.scatter(spikes.index, monthly_qty.loc[spikes.index], color='crimson', s=60, label='Cambio >25%')
plt.xticks(rotation=45)
plt.ylabel('Cantidad mensual')
plt.title('Ventas mensuales (Quantity)')
plt.legend()
plt.tight_layout()
plt.show()

spikes.to_frame(name='pct_change')


## Top productos


In [ ]:
top_products = df['Description'].value_counts().head(20)
plt.figure(figsize=(8, 6))
sns.barplot(x=top_products.values, y=top_products.index, palette='viridis')
plt.xlabel('Ventas (conteo)')
plt.ylabel('Producto')
plt.title('Top 20 productos')
plt.tight_layout()
plt.show()

top_products.to_frame(name='count')


## Canastas y tamanos de ticket


In [ ]:
basket = df.groupby('InvoiceNo')['Description'].apply(list)
basket_size = basket.apply(len)
print(f"Transacciones: {len(basket)} | Items unicos: {df['Description'].nunique()} | Ticket mediano: {basket_size.median():.0f}")

plt.figure(figsize=(6,4))
sns.histplot(basket_size, bins=30, kde=True)
plt.xlabel('Items por factura')
plt.title('Distribucion de tamano de canasta')
plt.tight_layout()
plt.show()

basket.head()


## Guardar dataset procesado


In [ ]:
PROCESSED_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(PROCESSED_PATH, index=False)
print(f"Guardado en {PROCESSED_PATH}")
